In [22]:
#%%capture
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.0/spark-sql-kafka-0-10_2.12-3.5.0.jar"
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10_2.12/3.5.0/spark-streaming-kafka-0-10_2.12-3.5.0.jar"

In [23]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [25]:
spark = SparkSession.builder.appName("MyApp3_ReadStreamFromKafka").getOrCreate()

lines = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "192.168.1.139:8097") \
  .option("subscribe", "input") \
  .option("startingOffsets", "latest") \
  .load()

In [26]:
lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [27]:
#Lab 2.1
schema = StructType() \
    .add("event_time", TimestampType()) \
    .add("message", StringType())

parsed = lines.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), schema).alias("data")) \
    .select("data.event_time", "data.message")

parsed.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- message: string (nullable = true)



In [28]:
words = parsed.select(
   explode(
       split(parsed.message, " ")
   ).alias("word"),
   col("event_time")
)

In [29]:
words.printSchema()

root
 |-- word: string (nullable = false)
 |-- event_time: timestamp (nullable = true)



In [41]:
#, "5 seconds"),
wordCounts = words.groupBy(
    window("event_Time", "60 seconds"),
    "word"
).count()

In [42]:
wordCounts.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- word: string (nullable = false)
 |-- count: long (nullable = false)



In [ ]:
#update

In [ ]:
if True:
    query = wordCounts \
      .writeStream \
      .trigger(processingTime='1 seconds') \
      .outputMode("update") \
      .format("console") \
      .option("truncate", False) \
      .start()
    
    query.awaitTermination()
#Try to use update

In [58]:
for q in spark.streams.active:
    q.stop()